## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Avarua,-21.21,-159.78,68.00,94,40,3.36,scattered clouds,CK,2020-11-08 14:14:38
1,1,Prince Rupert,54.32,-130.32,39.20,92,40,6.93,scattered clouds,CA,2020-11-08 14:18:09
2,2,Suntar,62.14,117.63,-2.45,91,0,3.89,clear sky,RU,2020-11-08 14:18:10
3,3,Ushuaia,-54.80,-68.30,51.80,53,40,17.22,scattered clouds,AR,2020-11-08 14:14:27
4,4,Tuatapere,-46.13,167.68,50.00,86,88,3.00,overcast clouds,NZ,2020-11-08 14:14:22


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
6,6,Vitoria Da Conquista,-14.87,-40.84,75.81,73,89,7.58,light rain,BR,2020-11-08 14:17:06
22,22,Namatanai,-3.67,152.43,79.92,80,90,3.85,overcast clouds,PG,2020-11-08 14:16:23
29,29,Mazatlan,23.22,-106.42,79.00,94,5,4.70,clear sky,MX,2020-11-08 14:18:13
36,36,Arraial Do Cabo,-22.97,-42.03,77.00,78,75,9.17,broken clouds,BR,2020-11-08 14:14:47
37,37,Mahebourg,-20.41,57.70,77.00,78,40,11.41,scattered clouds,MU,2020-11-08 14:14:41
38,38,Tiarei,-17.53,-149.33,75.20,83,20,3.36,few clouds,PF,2020-11-08 14:18:14
41,41,Boyuibe,-20.42,-63.28,80.60,44,40,11.32,smoke,BO,2020-11-08 14:18:15
42,42,Inhambane,-23.86,35.38,84.20,79,20,20.80,few clouds,MZ,2020-11-08 14:18:15
43,43,Acapulco,16.86,-99.89,77.00,88,20,3.40,few clouds,MX,2020-11-08 14:13:49
46,46,Kapaa,22.08,-159.32,78.80,78,40,9.17,scattered clouds,US,2020-11-08 14:14:37


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                214
City                   214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
Country                214
Date                   214
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Vitoria Da Conquista,BR,75.81,light rain,-14.87,-40.84,
22,Namatanai,PG,79.92,overcast clouds,-3.67,152.43,
29,Mazatlan,MX,79.00,clear sky,23.22,-106.42,
36,Arraial Do Cabo,BR,77.00,broken clouds,-22.97,-42.03,
37,Mahebourg,MU,77.00,scattered clouds,-20.41,57.70,
38,Tiarei,PF,75.20,few clouds,-17.53,-149.33,
41,Boyuibe,BO,80.60,smoke,-20.42,-63.28,
42,Inhambane,MZ,84.20,few clouds,-23.86,35.38,
43,Acapulco,MX,77.00,few clouds,16.86,-99.89,
46,Kapaa,US,78.80,scattered clouds,22.08,-159.32,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, IndentationError, IndentationError):
        print("Hotel not found... skipping.")
            

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")

hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))